# Prueba Técnica

Nuestra empresa conecta proveedores de productos de viajes y distribuidores de viajes. 
Objetivo: limpiar y preparar un conjunto de datos antes de cargarlo en un data warehouse para su análisis. 
Dataset: contiene información como reservas afectadas, solicitantes, aprobadores de estas solicitudes, precios, moneda, motivo de solicitud, estado y datos del cliente. 
Requisitos: 
- No debe haber más de dos peticiones para la misma reserva y misma cantidad. 
- Todas las peticiones deben tener a una persona en Authorized by. 
- Todos los correos deben tener el formato correcto. 
- Todas las cantidades deben cambiarse a euros y hay que borrar la columna Amount_comges_in_EUR 
- La columna “reason” es obligatoria, la “reason 2”, recomendada. 
- Hacer una breve visualización de los datos para entenderlos mejor.

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [19]:
# Importar librerías necesarias

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np


# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as st
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas del DataFrame

In [21]:
# Carga del Data Frame

df = pd.read_excel("requests.xlsx")
df.head(3)

,Booking,Request date,Requested by,Authorized by,Department,Currency,Amount,Reason,Reason 2,Status,CustomerShortname,CustomerRegion,Amount COMGES in EUR
0,100/1000000,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,74.820,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT82,Region 4,9.613136
1,100/1000001,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,424.365,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT141,Region 1,54.523904
2,100/1000002,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,104.005,BOOKING_TECHNICAL_ISSUE,PRICE DISCREPANCY ACROSS BOOKING PROCESS,Applied,CLIENT141,Region 1,13.362927


In [22]:
# Realizamos una exploración inicial de los datos para identificar posibles problemas, como valores nulos, atípicos o datos faltantes en las columnas relevantes
# Solicitamos visión general del DF
display(df.head(3))
display(df.tail(3))
display(df.sample(3))

,Booking,Request date,Requested by,Authorized by,Department,Currency,Amount,Reason,Reason 2,Status,CustomerShortname,CustomerRegion,Amount COMGES in EUR
0,100/1000000,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,74.820,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT82,Region 4,9.613136
1,100/1000001,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,424.365,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT141,Region 1,54.523904
2,100/1000002,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,104.005,BOOKING_TECHNICAL_ISSUE,PRICE DISCREPANCY ACROSS BOOKING PROCESS,Applied,CLIENT141,Region 1,13.362927


,Booking,Request date,Requested by,Authorized by,Department,Currency,Amount,Reason,Reason 2,Status,CustomerShortname,CustomerRegion,Amount COMGES in EUR
239397,100/1239397,2024-07-19 00:00:00,user7@hotelbeds.com,approver40@hotelbeds.com,Sales,CAD,345.63,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Requested,CLIENT283,Region 3,235.731824
239398,100/1239398,2024-07-19 00:00:00,user7@hotelbeds.com,approver40@hotelbeds.com,Sales,CAD,78.75,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Requested,CLIENT283,Region 3,53.710271
239399,100/1239399,2024-07-22 00:00:00,user44@hotelbeds.com,approver42@hotelbeds.com,Sales,IDR,7003421.00,OTHERS,PRICE MATCH,Requested,CLIENT120,Region 1,399.281018


,Booking,Request date,Requested by,Authorized by,Department,Currency,Amount,Reason,Reason 2,Status,CustomerShortname,CustomerRegion,Amount COMGES in EUR
81716,100/1081716,2024-03-21 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,11.17,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT3,Region 3,10.428045
185507,100/1185507,2024-05-30 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,10.06,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT2,Region 2,9.391775
177732,100/1177732,2024-05-30 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,3.51,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT1,Region 1,3.276852


In [23]:
df.shape

(239400, 13)

In [24]:
print(f"El número de filas es {df.shape[0]} y el número de columnas es {df.shape[1]}")

El número de filas es 239400 y el número de columnas es 13


In [25]:
# Los nombres de las columnas son correctos y claros, pero como buena práctica de Python y Pandas, cambiamos los espacios por guiones bajos
nuevas_columnas = {col: col.replace(' ','_') for col in df.columns}
nuevas_columnas

{'Booking': 'Booking',
 'Request date': 'Request_date',
 'Requested by': 'Requested_by',
 'Authorized by': 'Authorized_by',
 'Department': 'Department',
 'Currency': 'Currency',
 'Amount': 'Amount',
 'Reason': 'Reason',
 'Reason 2': 'Reason_2',
 'Status': 'Status',
 'CustomerShortname': 'CustomerShortname',
 'CustomerRegion': 'CustomerRegion',
 'Amount COMGES in EUR': 'Amount_COMGES_in_EUR'}

In [26]:
df.rename(columns = nuevas_columnas, inplace=True)
df.columns

Index(['Booking', 'Request_date', 'Requested_by', 'Authorized_by',
       'Department', 'Currency', 'Amount', 'Reason', 'Reason_2', 'Status',
       'CustomerShortname', 'CustomerRegion', 'Amount_COMGES_in_EUR'],
      dtype='object')

In [27]:
# Solicitamos información general del DF
df.info() # observamos que hay nulos en algunas columnas y que los datos son todos de tipo texto o numéricos decimales.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239400 entries, 0 to 239399
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Booking               239400 non-null  object 
 1   Request_date          239400 non-null  object 
 2   Requested_by          239400 non-null  object 
 3   Authorized_by         239398 non-null  object 
 4   Department            239400 non-null  object 
 5   Currency              239400 non-null  object 
 6   Amount                239400 non-null  float64
 7   Reason                239397 non-null  object 
 8   Reason_2              239316 non-null  object 
 9   Status                239400 non-null  object 
 10  CustomerShortname     239400 non-null  object 
 11  CustomerRegion        239400 non-null  object 
 12  Amount_COMGES_in_EUR  239392 non-null  float64
dtypes: float64(2), object(11)
memory usage: 23.7+ MB


In [28]:
# Utilizamos un .describe() para obtener una visión general de las principales estadísticas descriptivas
# Transponemos los resultados con .T para tener una mejor lectura
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Amount,239400.0,1408.656182,184094.647958,-219.9,1.89,2.730000,4.580000,38000000.00
Amount_COMGES_in_EUR,239392.0,12.983835,99.618487,-219.9,1.77,2.557998,4.275778,18356.96


In [29]:
# Calculamos cuántos valores nulos hay en las columnas que los tienen
print("Valores nulos en el df:")
df.isnull().sum()

Valores nulos en el df:


Booking                  0
Request_date             0
Requested_by             0
Authorized_by            2
Department               0
Currency                 0
Amount                   0
Reason                   3
Reason_2                84
Status                   0
CustomerShortname        0
CustomerRegion           0
Amount_COMGES_in_EUR     8
dtype: int64

In [30]:
# Utilizamos .duplicated() para identificar filas duplicadas en el DF
print("Serie booleana de filas duplicadas (considerando todas las columnas):")
df.duplicated()

Serie booleana de filas duplicadas (considerando todas las columnas):


0         False
1         False
2         False
3         False
4         False
          ...  
239395    False
239396    False
239397    False
239398    False
239399    False
Length: 239400, dtype: bool

In [31]:
# Obtenemos el conteo de filas duplicadas
print("\nNúmero total de filas duplicadas en el DataFrame:")
df.duplicated().sum()


Número total de filas duplicadas en el DataFrame:


2

In [32]:
# Comprobamos el número el valores duplicados para la columna 'Booking', que representa un identificador único para cada request 
# y quizá en algún momento interesa eliminar duplicados de esta columna en concreto
print("\nNúmero de valores duplicados en la columna 'Booking':")
df.duplicated(subset = "Booking").sum()


Número de valores duplicados en la columna 'Booking':


7

In [33]:
# Para comprobar duplicados y verificar que existen, filtramos solo las filas con Booking duplicado
duplicados_Booking = df[df.duplicated(subset="Booking", keep=False)]

# Ordenamos los duplicados de menor a mayor por Booking para facilitar la visualización
duplicados_Booking = duplicados_Booking.sort_values(by="Booking", ascending=True)

# Queremos una muestra de los 7 duplicados ya ordenados
muestra_duplicados = duplicados_Booking.head(7)

print(muestra_duplicados)

            Booking         Request_date          Requested_by  \
12      100/1000012  2024-01-02 00:00:00  user39@hotelbeds.com   
15      100/1000012  2024-01-02 00:00:00  user39@hotelbeds.com   
51657   100/1051657  2024-02-23 00:00:00  user23@hotelbeds.com   
51665   100/1051657  2024-02-23 00:00:00  user23@hotelbeds.com   
51740   100/1051740  2024-02-23 00:00:00  user23@hotelbeds.com   
51751   100/1051740  2024-02-23 00:00:00  user23@hotelbeds.com   
151675  100/1151675  2024-05-30 00:00:00  user23@hotelbeds.com   

                   Authorized_by Department Currency  Amount      Reason  \
12      approver17@hotelbeds.com      Sales      GBP  138.54  RATE_ERROR   
15      approver17@hotelbeds.com      Sales      GBP  139.55  RATE_ERROR   
51657   approver25@hotelbeds.com      Sales      USD    1.83      OTHERS   
51665   approver25@hotelbeds.com      Sales      USD    2.00      OTHERS   
51740   approver25@hotelbeds.com      Sales      USD    2.99      OTHERS   
51751   approve

In [34]:
def check_duplicates_in_column(df, column_name):
    """
    Checks for duplicated values in a specified column of a DataFrame
    and returns the duplicated values.

    Args:
    - df (pd.DataFrame): The DataFrame to check.
    - column_name (str): The column to check for duplicates.

    Returns:
    - Series: A Series containing the duplicated values.
    """

    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")
    
    # Find duplicated values
    duplicated_values = df[column_name][df[column_name].duplicated()].unique()
    
    if len(duplicated_values) > 0:
        print(f"Duplicated values in column '{column_name}': {duplicated_values}")
        print('***' * 50)
        print("Total duplicated: ", df[[column_name]].duplicated().sum())
    else:
        print(f"No duplicated values found in column '{column_name}'.")
    
    return duplicated_values

In [35]:
check_duplicates_in_column(df,'Booking')

Duplicated values in column 'Booking': ['100/1000012' '100/1051657' '100/1051740' '100/1151675' '100/1200589'
 '100/1239393']
******************************************************************************************************************************************************
Total duplicated:  7


array(['100/1000012', '100/1051657', '100/1051740', '100/1151675',
       '100/1200589', '100/1239393'], dtype=object)

In [36]:
# Filtrar filas cuyo 'Booking' está duplicado
duplicated_bookings = df[df['Booking'].duplicated(keep=False)]

# Mostrarlas (ordenadas por 'Booking' para comparar fácilmente)
duplicated_bookings.sort_values('Booking')

,Booking,Request_date,Requested_by,Authorized_by,Department,Currency,Amount,Reason,Reason_2,Status,CustomerShortname,CustomerRegion,Amount_COMGES_in_EUR
12,100/1000012,2024-01-02 00:00:00,user39@hotelbeds.com,approver17@hotelbeds.com,Sales,GBP,138.54,RATE_ERROR,TARIFF ERROR PROTOCOL,Applied,CLIENT62,Region 5,163.507612
15,100/1000012,2024-01-02 00:00:00,user39@hotelbeds.com,approver17@hotelbeds.com,Sales,GBP,139.55,RATE_ERROR,TARIFF ERROR PROTOCOL,Applied,CLIENT62,Region 5,164.699634
51657,100/1051657,2024-02-23 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,1.83,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT1,Region 1,1.708444
51665,100/1051657,2024-02-23 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,2.00,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT1,Region 1,1.867152
51740,100/1051740,2024-02-23 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,2.99,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT1,Region 1,2.791392
51751,100/1051740,2024-02-23 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,1.28,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT2,Region 2,1.194977
151675,100/1151675,2024-05-30 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,3.09,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT1,Region 1,2.884750
151683,100/1151675,2024-05-30 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,2.62,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT3,Region 3,2.445969
200589,100/1200589,2024-07-05 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,9.69,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT2,Region 2,9.046352
200597,100/1200589,2024-07-05 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,9.69,OTHERS,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT1,Region 1,4.359800


In [40]:
# Ver conteo de repeticiones por booking
Booking_counts = df['Booking'].value_counts()
Booking_counts[Booking_counts > 1]  # Solo los que aparecen más de una vez

Booking
100/1239393    3
100/1200589    2
100/1051740    2
100/1000012    2
100/1051657    2
100/1151675    2
Name: count, dtype: int64

In [41]:
Booking_counts[Booking_counts > 2]

Booking
100/1239393    3
Name: count, dtype: int64

In [42]:
# Filas que son 100% idénticas (en todos los campos)
df[df.duplicated(keep=False)]

,Booking,Request_date,Requested_by,Authorized_by,Department,Currency,Amount,Reason,Reason_2,Status,CustomerShortname,CustomerRegion,Amount_COMGES_in_EUR
239392,100/1239393,2024-07-19 00:00:00,user72@hotelbeds.com,approver39@hotelbeds.com,Sales,USD,144.54,OTHERS,NaN,Requested,CLIENT56,Region 4,134.939084
239393,100/1239393,2024-07-19 00:00:00,user72@hotelbeds.com,approver39@hotelbeds.com,Sales,USD,144.54,OTHERS,NaN,Requested,CLIENT56,Region 4,134.939084
239394,100/1239393,2024-07-19 00:00:00,user72@hotelbeds.com,approver39@hotelbeds.com,Sales,USD,144.54,OTHERS,NaN,Requested,CLIENT56,Region 4,134.939084


In [44]:
# Lmpieza de duplicados exactos si hay mas de dos peticiones con mismo Booking
over_limit = df['Booking'].value_counts()
over_limit = over_limit[over_limit > 2].index.tolist()

# ver filas
df[df['Booking'].isin(over_limit)].sort_values('Booking')

,Booking,Request_date,Requested_by,Authorized_by,Department,Currency,Amount,Reason,Reason_2,Status,CustomerShortname,CustomerRegion,Amount_COMGES_in_EUR
239392,100/1239393,2024-07-19 00:00:00,user72@hotelbeds.com,approver39@hotelbeds.com,Sales,USD,144.54,OTHERS,NaN,Requested,CLIENT56,Region 4,134.939084
239393,100/1239393,2024-07-19 00:00:00,user72@hotelbeds.com,approver39@hotelbeds.com,Sales,USD,144.54,OTHERS,NaN,Requested,CLIENT56,Region 4,134.939084
239394,100/1239393,2024-07-19 00:00:00,user72@hotelbeds.com,approver39@hotelbeds.com,Sales,USD,144.54,OTHERS,NaN,Requested,CLIENT56,Region 4,134.939084


In [45]:
# Eliminar filas si hay mas de 2 Booking request con mismo Booking
df = df.sort_values('Booking').groupby('Booking').head(2).reset_index(drop=True)

In [46]:
check_duplicates_in_column(df,'Booking')

Duplicated values in column 'Booking': ['100/1000012' '100/1051657' '100/1051740' '100/1151675' '100/1200589'
 '100/1239393']
******************************************************************************************************************************************************
Total duplicated:  6


array(['100/1000012', '100/1051657', '100/1051740', '100/1151675',
       '100/1200589', '100/1239393'], dtype=object)

In [47]:
df_sindup = df.drop_duplicates(subset= ['Booking', 'Amount'], keep='first')

In [48]:
df_sindup.head()

,Booking,Request_date,Requested_by,Authorized_by,Department,Currency,Amount,Reason,Reason_2,Status,CustomerShortname,CustomerRegion,Amount_COMGES_in_EUR
0,100/1000000,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,74.820,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT82,Region 4,9.613136
1,100/1000001,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,424.365,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT141,Region 1,54.523904
2,100/1000002,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,104.005,BOOKING_TECHNICAL_ISSUE,PRICE DISCREPANCY ACROSS BOOKING PROCESS,Applied,CLIENT141,Region 1,13.362927
3,100/1000003,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,39.050,BOOKING_OPERATIONAL_ISSUE,BOOKOUT – HOTEL SOLD OUT BEFORE ARRIVAL,Applied,CLIENT141,Region 1,5.017281
4,100/1000004,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,981.460,BOOKING_OPERATIONAL_ISSUE,RESERVATION NOT FOUND ON SPOT,Applied,CLIENT82,Region 4,126.101425


In [49]:
df = df_sindup

In [50]:
# comprobar que Authorized by no tiene nulos
print(f"El dataset tiene {df.isnull().sum()} valores nulos por columna")

El dataset tiene Booking                  0
Request_date             0
Requested_by             0
Authorized_by            2
Department               0
Currency                 0
Amount                   0
Reason                   3
Reason_2                82
Status                   0
CustomerShortname        0
CustomerRegion           0
Amount_COMGES_in_EUR     8
dtype: int64 valores nulos por columna


In [51]:
print(f"Filas con 'Authorized_by' faltante: {df['Authorized_by'].isnull().sum()}")


Filas con 'Authorized_by' faltante: 2


In [52]:
df[df['Authorized_by'].isnull()].sort_values('Booking')

,Booking,Request_date,Requested_by,Authorized_by,Department,Currency,Amount,Reason,Reason_2,Status,CustomerShortname,CustomerRegion,Amount_COMGES_in_EUR
471,100/1000471,2024-01-16 00:00:00,user88hotelbeds.com,NaN,Sales,EUR,211.00,BOOKING_OPERATIONAL_ISSUE,NaN,Applied,CLIENT532,Region 2,211.000000
239394,100/1239395,2024-07-19 00:00:00,user16@hotelbeds.com,NaN,Sales,BRL,1105.44,NaN,RESERVATION DISCREPANCIES,Requested,CLIENT620,Region 2,184.763497


In [53]:
df['Authorized_by'].unique()

array(['approver21@hotelbeds.com', 'approver43@hotelbeds.com',
       'approver14@hotelbeds.com', 'approver37@hotelbeds.com',
       'approver17@hotelbeds.com', 'approver35@hotelbeds.com',
       'approver7@hotelbeds.com', 'approver2@hotelbeds.com',
       'approver42@hotelbeds.com', 'approver12@hotelbeds.com',
       'approver31@hotelbeds.com', 'approver9@hotelbeds.com',
       'approver8@hotelbeds.com', 'approver22@hotelbeds.com',
       'approver39@hotelbeds.com', 'approver10@hotelbeds.com',
       'approver28@hotelbeds.com', 'approver4@hotelbeds.com',
       'approver38@hotelbeds.com', 'approver16@hotelbeds.com', nan,
       'approver25@hotelbeds.com', 'approver40@hotelbeds.com',
       'approver26@hotelbeds.com', 'approver15@hotelbeds.com',
       'approver29@hotelbeds.com', 'approver20@hotelbeds.com',
       'approver6@hotelbeds.com', 'approver24@hotelbeds.com',
       'approver30@hotelbeds.com', 'approver5@hotelbeds.com',
       'approver34@hotelbeds.com', 'approver19@hotelbeds.

In [54]:
# se reemplaza por Unknown
df['Authorized_by'] = df['Authorized_by'].fillna('Unknown')

In [55]:
df['Authorized_by'].unique()

array(['approver21@hotelbeds.com', 'approver43@hotelbeds.com',
       'approver14@hotelbeds.com', 'approver37@hotelbeds.com',
       'approver17@hotelbeds.com', 'approver35@hotelbeds.com',
       'approver7@hotelbeds.com', 'approver2@hotelbeds.com',
       'approver42@hotelbeds.com', 'approver12@hotelbeds.com',
       'approver31@hotelbeds.com', 'approver9@hotelbeds.com',
       'approver8@hotelbeds.com', 'approver22@hotelbeds.com',
       'approver39@hotelbeds.com', 'approver10@hotelbeds.com',
       'approver28@hotelbeds.com', 'approver4@hotelbeds.com',
       'approver38@hotelbeds.com', 'approver16@hotelbeds.com', 'Unknown',
       'approver25@hotelbeds.com', 'approver40@hotelbeds.com',
       'approver26@hotelbeds.com', 'approver15@hotelbeds.com',
       'approver29@hotelbeds.com', 'approver20@hotelbeds.com',
       'approver6@hotelbeds.com', 'approver24@hotelbeds.com',
       'approver30@hotelbeds.com', 'approver5@hotelbeds.com',
       'approver34@hotelbeds.com', 'approver19@hote

In [57]:
# Unificar estilo de los emails
# Para 'Authorized_by'
df['Authorized_by'] = df['Authorized_by'].str.replace('.Com', '.com', regex=False)
df['Authorized_by'] = df['Authorized_by'].str.replace('.COM', '.com', regex=False)
df['Authorized_by'] = df['Authorized_by'].str.replace(r'\.Hotelbeds\.com', '@Hotelbeds.com', regex=True)
df['Authorized_by'] = df['Authorized_by'].str.lower()

# Para 'Requested_by'
df['Requested_by'] = df['Requested_by'].str.replace('.Com', '.com', regex=False)
df['Requested_by'] = df['Requested_by'].str.replace('.COM', '.com', regex=False)
df['Requested_by'] = df['Requested_by'].str.replace(r'\.Hotelbeds\.com', '@Hotelbeds.com', regex=True)
df['Requested_by'] = df['Requested_by'].str.lower()

In [58]:
df.head(25)

,Booking,Request_date,Requested_by,Authorized_by,Department,Currency,Amount,Reason,Reason_2,Status,CustomerShortname,CustomerRegion,Amount_COMGES_in_EUR
0,100/1000000,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,74.820000,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT82,Region 4,9.613136
1,100/1000001,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,424.365000,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT141,Region 1,54.523904
2,100/1000002,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,104.005000,BOOKING_TECHNICAL_ISSUE,PRICE DISCREPANCY ACROSS BOOKING PROCESS,Applied,CLIENT141,Region 1,13.362927
3,100/1000003,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,39.050000,BOOKING_OPERATIONAL_ISSUE,BOOKOUT – HOTEL SOLD OUT BEFORE ARRIVAL,Applied,CLIENT141,Region 1,5.017281
4,100/1000004,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,981.460000,BOOKING_OPERATIONAL_ISSUE,RESERVATION NOT FOUND ON SPOT,Applied,CLIENT82,Region 4,126.101425
5,100/1000005,2024-01-02 00:00:00,user46@hotelbeds.com,approver43@hotelbeds.com,Sales,CNY,158.000000,OTHERS,PRICE MATCH,Applied,CLIENT429,Region 3,20.300394
6,100/1000006,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,USD,66.430000,BOOKING_OPERATIONAL_ISSUE,RESERVATION NOT FOUND ON SPOT,Applied,CLIENT176,Region 5,62.017458
7,100/1000007,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,USD,66.430000,BOOKING_OPERATIONAL_ISSUE,RESERVATION NOT FOUND ON SPOT,Applied,CLIENT176,Region 5,62.017458
8,100/1000008,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,USD,44.200000,BOOKING_OPERATIONAL_ISSUE,RESERVATION NOT FOUND ON SPOT,Applied,CLIENT176,Region 5,41.264062
9,100/1000009,2024-01-02 00:00:00,user101@hotelbeds.com,approver14@hotelbeds.com,Sales,AED,404.470000,OTHERS,PRICE MATCH,Applied,CLIENT38,Region 2,102.804784


In [60]:
# revisar Amount, dejar en valores de euros y eliminar la última columna
df['Currency'].unique()

array(['CNY', 'USD', 'AED', 'HKD', 'GBP', 'EUR', 'BRL', 'UDS', 'JPY',
       'MXN', 'CYN', 'IDR', 'AUD', 'KRW', 'NZD', 'ZAR', 'INR', 'KWR',
       'THB', 'CAD', 'COP', 'SAR', 'CHF', 'SGD', 'NOK', 'PHP'],
      dtype=object)

In [63]:
# Reemplazar la columna 'Amount' con los valores en EUR
df['Amount'] = df['Amount_COMGES_in_EUR']

#Borrar la columna 'Amount_COMGES_in_EUR'
df = df.drop(columns=['Amount_COMGES_in_EUR'])

# Renombrar 'Amount' a algo más claro
df.rename(columns={'Amount': 'Amount_eur'}, inplace=True)

In [64]:
df.head()

,Booking,Request_date,Requested_by,Authorized_by,Department,Currency,Amount_eur,Reason,Reason_2,Status,CustomerShortname,CustomerRegion
0,100/1000000,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,9.613136,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT82,Region 4
1,100/1000001,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,54.523904,BOOKING_OPERATIONAL_ISSUE,RESERVATION DISCREPANCIES,Applied,CLIENT141,Region 1
2,100/1000002,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,13.362927,BOOKING_TECHNICAL_ISSUE,PRICE DISCREPANCY ACROSS BOOKING PROCESS,Applied,CLIENT141,Region 1
3,100/1000003,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,5.017281,BOOKING_OPERATIONAL_ISSUE,BOOKOUT – HOTEL SOLD OUT BEFORE ARRIVAL,Applied,CLIENT141,Region 1
4,100/1000004,2024-01-02 00:00:00,user94@hotelbeds.com,approver21@hotelbeds.com,Sales,CNY,126.101425,BOOKING_OPERATIONAL_ISSUE,RESERVATION NOT FOUND ON SPOT,Applied,CLIENT82,Region 4


In [65]:
# Ver los nuloe de 'Reason'
missing_reason = df[df['Reason'].isnull()]
print("Filas de 'Reason' con nulos:")
print(missing_reason[['Reason', 'Reason_2']])

Filas de 'Reason' con nulos:
       Reason                   Reason_2
96323     NaN         CANCELLATION WAIVE
96324     NaN         CANCELLATION WAIVE
239394    NaN  RESERVATION DISCREPANCIES


In [67]:
# Rellenar 'Reason' con 'Reason_2' solo si 'Reason' es NaN
df['Reason'] = df['Reason'].fillna(df['Reason_2'])

In [68]:
print("Nulos en 'Reason' antes:", missing_reason.shape[0])
print("Nulos en 'Reason' después:", df['Reason'].isnull().sum())

Nulos en 'Reason' antes: 3
Nulos en 'Reason' después: 0


In [71]:
# Guardar dataframe limpio en CSV
df.to_csv(
    "requests_clean.csv",
    index=False,
    encoding="utf-8"
)